<font face="Calibri" size=5>Credit Valuation Adjustment on Derivative Instruments </font>

This is a Python 3 version of a CVA project.
- The project was done in a 5 person team, over multiple weeks, in a Financial Engineering course.
- The original code was in R, and much of the visual analysis was in Excel.
- This is my conversion, and revision of the R code - done individually.

<font color="blue">Goal: A report describing applied methodologies, as well as CVA calculations for different types of derivative insturments.</font>


<hr>

<font size=2>Global Imports</font>

In [9]:
import pandas as pd

---
<hr>

<font face="Calibri" size=4>Part 0. Description of the Derivative Instruments
</font>

- 1. Buy **EUR** sell PLN FX Forward  
(maturity: 1 year, notional: 100 000 EUR; strike: 4.3930)


- 2. Receiver (We receive fixed rate) PLN WIBOR 3M IRS  
(maturity: 3 years, notional: 500 000 PLN; fixed rate: 2.2144% - paid annually)


- 3. EUR/PLN CIRS, paid rate: fixed rate (-0,0575%) in EUR,  
received rate WIBOR 3M, notional exchange at the beginning as well as end of the contract  
(maturity: 3 years, notional: 100 000 EUR/ 430 000 PLN)

<font face="Calibri" size=4>Part I. Analysis of Instruments
</font>
- ### Mechanics
- ### Payoff formulars & Payoff Profiles
- ### Valuation Formulas

<font face="Calibri" size=4>Part II. Implied Defauly Probability
</font>
- ### CDS Contracts
  - Mechanics
  - Valuation
- ### Implying Probability of Default from a CDS spread
  - CDS bootsrapping
  - CDS bootstrapping from sample market quotes

_Bootsrapping is any test of metric that relies on random sampling with replacement. It allows assigning measures of accuracy __(defined in terms of bias, variance, confidence intervals, prediction error, etc)__ to sample estimates. This techinique allows estimation of the sampling distribution of almost any statistic, using ranomd sampling methods. (Wikipedia)_

[+] Data

In [29]:
# As CDS spreads are not readily available for free,
# this market data was supplied as part of the project.
# The data, oringinal in .xlsx, has been copied into Pandas DataFrames.

# Our CDS data has tenors of .5 years to 10 yrs. 
# We add a column for this length, in years
Market_CDS_Spreads = pd.DataFrame({"CDS_DB":["DB6MEUSM=R","DB1YEUSM=R",
                                          "DB2YEUSM=R","DB3YEUSM=R",
                                          "DB4YEUSM=R","DB5YEUSM=R",
                                          "DB7YEUSM=R","DB10YEUSM=R"],
                                   "CDS_Tenor":[0.5, 1,
                                                2, 3,
                                                4, 5,
                                                7, 10],
                                "Close" :[35.88,39.40,69.35,96.70,
                                          123.33,152.07,196.58,223.30]})

Market_EURPLN_Xr = 4.3000

# The index in this DataFrame represents the number of months into the future
Discount_Factors = pd.DataFrame([
    [1.0000,1.0000,4.3000], [1.0004,0.9983,4.3090], [1.0006,0.9972,4.3149],
    [1.0009,0.9958,4.3218], [1.0012,0.9944,4.3292], [1.0014,0.9929,4.3367],
    [1.0017,0.9915,4.3444], [1.0020,0.9900,4.3521], [1.0022,0.9885,4.3599],
    [1.0025,0.9869,4.3678], [1.0027,0.9853,4.3760], [1.0030,0.9837,4.3844],
    [1.0032,0.9820,4.3930], [1.0034,0.9802,4.4020], [1.0036,0.9783,4.4112],
    [1.0038,0.9765,4.4202], [1.0039,0.9747,4.4288], [1.0041,0.9730,4.4373],
    [1.0042,0.9713,4.4456], [1.0043,0.9696,4.4539], [1.0043,0.9678,4.4621],
    [1.0044,0.9661,4.4703], [1.0044,0.9643,4.4785], [1.0043,0.9626,4.4867],
    [1.0043,0.9607,4.4950], [1.0042,0.9589,4.5034], [1.0041,0.9570,4.5119],
    [1.0040,0.9551,4.5205], [1.0039,0.9531,4.5291], [1.0037,0.9511,4.5378],
    [1.0035,0.9490,4.5467], [1.0033,0.9469,4.5558], [1.0030,0.9448,4.5650],
    [1.0027,0.9426,4.5742], [1.0024,0.9405,4.5831], [1.0021,0.9384,4.5919],
    [1.0017,0.9362,4.6007], [1.0014,0.9341,4.6096], [1.0010,0.9319,4.6185],
    [1.0006,0.9298,4.6272], [1.0001,0.9277,4.6359], [0.9997,0.9256,4.6444],
    [0.9992,0.9234,4.6529], [0.9987,0.9213,4.6615], [0.9982,0.9191,4.6702],
    [0.9977,0.9169,4.6788], [0.9972,0.9148,4.6875], [0.9966,0.9126,4.6961],
    [0.9961,0.9104,4.7048], [0.9955,0.9081,4.7136], [0.9949,0.9059,4.7223],
    [0.9943,0.9037,4.7311], [0.9937,0.9014,4.7400], [0.9930,0.8992,4.7488],
    [0.9924,0.8969,4.7577], [0.9917,0.8947,4.7665], [0.9910,0.8924,4.7754],
    [0.9903,0.8901,4.7842], [0.9896,0.8878,4.7932], [0.9889,0.8855,4.8021],
    [0.9881,0.8832,4.8110]],
    columns = ["EUR_DF","PLN_DF","EUR/PLN"])

# for ease of use, we also add timespan in years to the DscFctr dframe
Discount_Factors["DF_Tenor"] = [tm/12 for tm in range(61)]
Discount_Factors = Discount_Factors[["DF_Tenor","EUR_DF","PLN_DF","EUR/PLN"]]

In [ ]:
trydf = pd.read_csv("")

In [30]:
print(Market_CDS_Spreads)
print("\nEUR/PLN :",Market_EURPLN_Xr,'\n')
print("---- Discount_Factors ----\n")
print(Discount_Factors.info())
print(Discount_Factors.head(),'\n',Discount_Factors.tail())

        CDS_DB  CDS_Tenor   Close
0   DB6MEUSM=R        0.5   35.88
1   DB1YEUSM=R        1.0   39.40
2   DB2YEUSM=R        2.0   69.35
3   DB3YEUSM=R        3.0   96.70
4   DB4YEUSM=R        4.0  123.33
5   DB5YEUSM=R        5.0  152.07
6   DB7YEUSM=R        7.0  196.58
7  DB10YEUSM=R       10.0  223.30

EUR/PLN : 4.3 

---- Discount_Factors ----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
DF_Tenor    61 non-null float64
EUR_DF      61 non-null float64
PLN_DF      61 non-null float64
EUR/PLN     61 non-null float64
dtypes: float64(4)
memory usage: 2.0 KB
None
   DF_Tenor  EUR_DF  PLN_DF  EUR/PLN
0  0.000000  1.0000  1.0000   4.3000
1  0.083333  1.0004  0.9983   4.3090
2  0.166667  1.0006  0.9972   4.3149
3  0.250000  1.0009  0.9958   4.3218
4  0.333333  1.0012  0.9944   4.3292 
     DF_Tenor  EUR_DF  PLN_DF  EUR/PLN
56  4.666667  0.9910  0.8924   4.7754
57  4.750000  0.9903  0.8901   4.7842
58  4.833333  0.9896  0.8878   4.793

<font face="Calibri" size=4>Part III. Risk Factors Simulation
</font>
- ### Simulation Methodolody
- ### Calibration to Market Data
- ### Simulation: 3 years, 1 month time step, $M$ simulations

<font face="Calibri" size=4>Part IV. Calculation of Expoected Exposure Profiles
</font>
- ### Expected Exposure (EE) profile for each derivative in question
- ### Identification and presentation of the key features of EE, per derivative
- ### Sensitivity analysis of volatility changes, on Expected Exposure

<font face="Calibri" size=4>Part V. CVA Calculation for Each Derivative
</font>
- ### CVA per derivative
- ### Sensitivity analysis of volatility changes on CVA